If your having problems running this on your machine. do the following...


brew install apache-spark

#setting env variable:

export SPARK_HOME="/usr/local/Cellar/apache-spark/2.1.0/libexec/"

export PYTHONPATH=$SPARK_HOME/python:$SPARK_HOME/python/build:$PYTHONPATH

export PYTHONPATH=$SPARK_HOME/python/lib/py4j-0.10.4-src.zip:$PYTHONPATH

export PYSPARK_DRIVER_PYTHON=jupyter

export PYSPARK_DRIVER_PYTHON_OPTS=notebook


pyspark


In [1]:
# MUST AT LEAST RUN THIS ONE TO GET THE DATA VARIABLE. OTHERS ARE OPTIONAL.
%matplotlib inline
from pyspark.sql import Row
from pyspark import SparkContext
import seaborn as sns
import re
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
from datetime import datetime
from pyspark.sql.functions import col,udf, unix_timestamp
from pyspark.sql.types import DateType
from pyspark.sql.functions import udf
from pyspark.sql.types import DataType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType
import locale

data = spark.read.csv("Nomis Solutions_Data Part I.csv", header=True, inferSchema=True)
data = data.where(data.Tier.isNotNull()) # remove bad data, 3 entries have 'null' for Tier
for column in data.columns: #fix column names
    data = data.withColumnRenamed(column, column.lower().replace(' ','_'))
data = data.withColumn('spread', (data.rate - data.competition_rate)/100) #calculate the spread
data = data.withColumn('rate', data.rate/100)
data = data.withColumn('competition_rate', data.competition_rate/100)
data = data.withColumn('cost_of_funds', data.cost_of_funds/100)
print(data.count())

# calculate out the NPV
def npv(loan, term, APR, cost_of_capital):
    interest = loan*APR # http://mathforum.org/dr.math/faq/faq.interest.html
    Cfs = []
    Cfs.append(-loan)
    for i in range(1,int(term/12)+1):
        Cfs.append(interest)
    Cfs[-1] += loan

    disc_Cfs = []
    for t,cf in enumerate(Cfs):
         disc_Cfs.append(cf / (1+cost_of_capital)**t)
    return sum(disc_Cfs)
    
print(npv(1000, 60, 0.07, 0.01))
npv = udf(npv, returnType=DoubleType())
data = data.withColumn('npv', npv("amount", "term", "rate", "cost_of_funds")) #.select("approve_date", "amount", "term", "rate", "cost_of_funds", "npv")

# calculate out the year
def year(stringDate):
    return datetime.strptime(stringDate, '%m/%d/%y').year
print(year("7/1/02"))
year = udf(year, returnType=IntegerType())
data = data.withColumn('year', year("approve_date"))
#calculate out the spread
data = data.withColumn('spread', data.rate - data.cost_of_funds)


208085
291.2058743595071
2002


In [2]:
data

DataFrame[tier: int, fico: int, approve_date: string, term: int, amount: double, previous_rate: string, car__type: string, competition_rate: double, outcome: int, rate: double, cost_of_funds: double, partner_bin: int, spread: double, npv: double, year: int]

In [3]:
data.show(1)

+----+----+------------+----+-------+-------------+---------+----------------+-------+-------------------+-------------+-----------+-------------------+-----------------+----+
|tier|fico|approve_date|term| amount|previous_rate|car__type|competition_rate|outcome|               rate|cost_of_funds|partner_bin|             spread|              npv|year|
+----+----+------------+----+-------+-------------+---------+----------------+-------+-------------------+-------------+-----------+-------------------+-----------------+----+
|   3| 695|      7/1/02|  72|35000.0|             |        N|          0.0625|      0|0.07490000000000001|     0.018388|          1|0.05651200000000001|11139.70700051541|2002|
+----+----+------------+----+-------+-------------+---------+----------------+-------+-------------------+-------------+-----------+-------------------+-----------------+----+
only showing top 1 row



In [4]:
data.describe('fico','term','competition_rate','outcome').show()

+-------+----------------+-----------------+--------------------+-------------------+
|summary|            fico|             term|    competition_rate|            outcome|
+-------+----------------+-----------------+--------------------+-------------------+
|  count|          208085|           208085|              208085|             208085|
|   mean|726.731407838143|56.80901554653146|0.048070775404278426|0.22003988754595477|
| stddev| 44.783564051538|11.20410261525545|0.005861487877211517| 0.4142742572366969|
|    min|             587|               36|0.029900000000000003|                  0|
|    max|             854|               72|              0.0645|                  1|
+-------+----------------+-----------------+--------------------+-------------------+



In [5]:
data.describe('rate','cost_of_funds', 'partner_bin', 'spread').show()

+-------+--------------------+--------------------+------------------+--------------------+
|summary|                rate|       cost_of_funds|       partner_bin|              spread|
+-------+--------------------+--------------------+------------------+--------------------+
|  count|              208085|              208085|            208085|              208085|
|   mean|0.056230491626017044|0.013288188254794572| 2.029670567316241| 0.04294230337121641|
| stddev|0.015469784190445417|0.002780538586589...|0.9110620679208952|0.015256583823850003|
|    min|              0.0245|              0.0102|                 1|0.009399999999999999|
|    max|              0.1553|0.021269999999999997|                 3|              0.1441|
+-------+--------------------+--------------------+------------------+--------------------+



In [6]:
data.groupBy('tier').count().sort('tier').show()

+----+------+
|tier| count|
+----+------+
|   1|100261|
|   2| 45472|
|   3| 39570|
|   4| 22782|
+----+------+



In [ ]:
# fico vs amount is interesting
x=data.select('tier','fico','amount','outcome', 'spread').toPandas()
sns.pairplot(x, hue='tier')

In [ ]:
# amount vs amount shows that the loans they are approving are low $ amount loans
# rate vs rate shows the loans they are approving are low interest rate loans
#x=data.select('tier','fico','amount','rate', 'cost_of_funds').toPandas()
x=data.select('fico','amount','rate', 'cost_of_funds', 'competition_rate', 'outcome', 'spread').toPandas()
sns.pairplot(x, hue='outcome')

$NPV=\sum_{n=1}^{n} \frac{Cn}{(1+r)^n} - Co$

In [ ]:
IRR

In [21]:
import pyspark.sql.functions as func
agg_npv = data.where(data.outcome==1).groupBy('year').sum('npv').toPandas()
agg_npv = agg_npv.sort_values('year',0)
#pd.options.display.float_format = '${:,.2f}'.format
agg_npv = agg_npv.set_index('year')
print(agg_npv)

          sum(npv)
year              
2002  1.913898e+07
2003  7.883488e+07
2004  7.628669e+07


In [23]:
cost_of_capital = data.where(data.outcome==1)#.where(data.approve_date.isNotNull())# .where(data.year == '2003')
cost_of_capital = cost_of_capital.toPandas()

for year in cost_of_capital['year'].unique():
    print(year, cost_of_capital[cost_of_capital['year']==year]['cost_of_funds'].mean())
avg_coc = [0.01689893351567463, 0.01177518111180385, 0.013632502898191337]

2002 0.01689893351567463
2003 0.01177518111180385
2004 0.013632502898191337


In [24]:
disc_Cfs = []
for t,cf in enumerate(agg_npv['sum(npv)']):
     disc_Cfs.append(cf / (1+avg_coc[t])**t)
print(sum(disc_Cfs))


171304873.154


In [ ]:
NPV for tier, FICO, approve date, term, amount

In [ ]:
EDA